<a href="https://colab.research.google.com/github/saliq-2/FashionMnist/blob/main/FashionMnistcnn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import torch
import torch.nn as nn
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

In [9]:
# Import data (ignore the warnings!)
from torch.utils.data import DataLoader
train_dataset = datasets.FashionMNIST('data/fashion',train=True, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_dataset = datasets.FashionMNIST('data/fashion',train=False, download=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
train_dataloader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=64, shuffle=True)

print(train_dataset)
print(dir(train_dataset))
print(train_dataset.train_data)

Dataset FashionMNIST
    Number of datapoints: 60000
    Root location: data/fashion
    Split: Train
    StandardTransform
Transform: Compose(
               ToTensor()
               Normalize(mean=(0.1307,), std=(0.3081,))
           )
['__add__', '__annotations__', '__class__', '__class_getitem__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__len__', '__lt__', '__module__', '__ne__', '__new__', '__orig_bases__', '__parameters__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__slots__', '__str__', '__subclasshook__', '__weakref__', '_check_exists', '_check_legacy_exist', '_format_transform_repr', '_is_protocol', '_load_data', '_load_legacy_data', '_repr_indent', 'class_to_idx', 'classes', 'data', 'download', 'extra_repr', 'mirrors', 'processed_folder', 'raw_folder', 'resources', 'root', 'target_transf

/usr/local/lib/python3.11/dist-packages/torchvision/datasets/mnist.py:76: UserWarning: train_data has been renamed data
  warnings.warn("train_data has been renamed data")


In [4]:
# Target classes
label_dict = {
 0: 'T-shirt/top',
 1: 'Trouser',
 2: 'Pullover',
 3: 'Dress',
 4: 'Coat',
 5: 'Sandal',
 6: 'Shirt',
 7: 'Sneaker',
 8: 'Bag',
 9: 'Ankle boot'
}

In [5]:
def accuracy(true_labels, predicted_labels):
    """
    Compute the classification accuracy for given labels and predictions
    :param true_labels: list of gold labels
    :param predicted_labels: list of predicted labels of same length as true_labels
    :return: a float between 0 and 1 describing how accurate the predicted labels are
    """
    accuracy = 0.0

    # YOUR CODE HERE
    true_labels = torch.tensor(true_labels)
    predicted_labels = torch.tensor(predicted_labels)

    correct = (true_labels == predicted_labels).sum().item()
    total = len(true_labels)

    accuracy = correct / total
    #raise NotImplementedError
    return accuracy

In [25]:
#cnn network arch conv-->max pool-->conv-->max pool-->fcn-->fcn
#Mimicking the vgg arch
class CNNFashion(nn.Module):
  def __init__(self):
    super(CNNFashion, self).__init__()
    self.cnn_layer1=nn.Conv2d(in_channels=1,out_channels=32,kernel_size=2,stride=1)
    self.cnn_layer2=nn.Conv2d(in_channels=32,out_channels=64,kernel_size=2,stride=1)
    self.max_pool=nn.MaxPool2d(kernel_size=2,stride=2)

    #Activation Function
    self.relu=nn.ReLU()
    #Linear Layers
    self.linear1=nn.Linear(6*6*64,1028)
    self.linear2=nn.Linear(1028,512)
    self.linear3=nn.Linear(512,128)
    self.linear4=nn.Linear(128,10)

  def forward(self,x):

    x=self.max_pool(self.relu(self.cnn_layer1(x)))

    x=self.max_pool(self.relu(self.cnn_layer2(x)))

    x=x.view(x.size(0),-1)

    x=self.relu(self.linear1(x))

    x=self.relu(self.linear2(x))

    x=self.relu(self.linear3(x))

    x=self.linear4(x)

    return x





In [26]:
#Defiining the model
model=CNNFashion()
criterion=nn.CrossEntropyLoss()
optimizer=torch.optim.Adam(model.parameters(),lr=0.001)


epochs=10




In [29]:
running_loss=0.0
#Training the model
for epoch in range(epochs):
  running_loss=0.0
  running_val_loss=0.0


  for images,labels in train_dataloader:
    model.train()


    optimizer.zero_grad()

    output=model(images)
    loss=criterion(output,labels)
    loss.backward()
    optimizer.step()
    running_loss+=loss.item()

  else:
    print(f"Epoch {epoch}--Training loss: {running_loss/len(train_dataloader)}")


    with torch.no_grad():
      model.eval
      accuracy_list=[]
      for images,labels in test_dataloader:
        output1=model(images)
        loss1=criterion(output1,labels)
        ps=torch.exp(output1)
        top_p,top_class=ps.topk(1,dim=1)
        equals=top_class==labels.view(*top_class.shape)
        accuracy_list.append(torch.mean(equals.type(torch.FloatTensor)))
        running_val_loss+=loss1.item()

    print(f"Validation Loss:{running_val_loss/len(test_dataloader)}--Validation accuracy: {torch.mean(torch.FloatTensor(accuracy_list))}")



Epoch 0--Training loss: 0.13625689079540212
Validation Loss:0.25601774325985815--Validation accuracy: 0.9130175113677979
Epoch 1--Training loss: 0.11101889425813198
Validation Loss:0.2805703359472144--Validation accuracy: 0.9120222926139832
Epoch 2--Training loss: 0.09312020986949576
Validation Loss:0.3161478436486736--Validation accuracy: 0.9082404375076294
Epoch 3--Training loss: 0.07520028901919881
Validation Loss:0.34340434198736386--Validation accuracy: 0.918292224407196
Epoch 4--Training loss: 0.06124903670207325
Validation Loss:0.3518589601561332--Validation accuracy: 0.9198845624923706
Epoch 5--Training loss: 0.05487783403701282
Validation Loss:0.3870709306874852--Validation accuracy: 0.9144108295440674
Epoch 6--Training loss: 0.04568017703751703
Validation Loss:0.39572787975334817--Validation accuracy: 0.9153065085411072
Epoch 7--Training loss: 0.03960295540230624
Validation Loss:0.425160486773131--Validation accuracy: 0.9123208522796631
Epoch 8--Training loss: 0.0343905433838